In [1]:
import polars as pl

In [2]:
!gsutil cp /home/eanegrin/buckets/b1/datasets/competencia_03_inflacion_adj_ipc.parquet /home/eanegrin/datasets/

Copying file:///home/eanegrin/buckets/b1/datasets/competencia_03_inflacion_adj_ipc.parquet...
- [1 files][861.8 MiB/861.8 MiB]                                                
Operation completed over 1 objects/861.8 MiB.                                    


In [3]:
base_path = '/home/eanegrin/'
# base_path = 'C:/Eugenio/Maestria/DMEyF/'

dataset_path = base_path + 'datasets/'

dataset_file = 'competencia_03_inflacion_adj_ipc.parquet'

In [18]:
df = pl.read_parquet(dataset_path + dataset_file)

In [19]:
# corrijo el tipo de estas 2:

df = df.with_columns([
    pl.col("tmobile_app").cast(pl.Int64),
    pl.col("cmobile_app_trx").cast(pl.Int64)
])

In [20]:
# elimino columnas con data drifting
rotas = ['cmobile_app_trx','tmobile_app','active_quarter','mplazo_fijo_pesos','minversion2']
df = df.select(pl.exclude(rotas))

In [21]:
# lista de columnas originales para operar despues
originales = df.select(pl.exclude(["numero_de_cliente", "foto_mes", "clase_ternaria"])).columns

In [22]:
# suma de columnas
df = df.with_columns([
    (pl.col("mcuenta_corriente_adicional").fill_null(0) +
     pl.col("mcuenta_corriente").fill_null(0) +
     pl.col("mcaja_ahorro_adicional").fill_null(0) +
     pl.col("mcaja_ahorro_dolares").fill_null(0) +
     pl.col("mcaja_ahorro").fill_null(0)).alias("suma_alt"),
    
    (pl.col("cseguro_vida").fill_null(0) +
     pl.col("cseguro_auto").fill_null(0) +
     pl.col("cseguro_vivienda").fill_null(0) +
     pl.col("cseguro_accidentes_personales").fill_null(0)).alias("cseguro_todos"),
    
    (pl.col("matm").fill_null(0) +
     pl.col("matm_other").fill_null(0)).alias("matm_total"),
    
    (pl.col("Visa_msaldototal").fill_null(0) +
     pl.col("Master_msaldototal").fill_null(0)).alias("tarjeta_msaldototal"),
    
    (pl.col("mactivos_margen").fill_null(0) +
     pl.col("mpasivos_margen").fill_null(0)).alias("map_margen"),
    
    (pl.col("ccuenta_corriente").fill_null(0) +
     pl.col("ccaja_ahorro").fill_null(0)).alias("cc_ca"),
    
    (pl.col("ctarjeta_visa").fill_null(0) +
     pl.col("ctarjeta_master").fill_null(0)).alias("ctarjeta_total"),
    
    (pl.col("ctarjeta_visa_transacciones").fill_null(0) +
     pl.col("ctarjeta_master_transacciones").fill_null(0)).alias("ctarjeta_total_transacciones"),
    
    (pl.col("mtarjeta_visa_consumo").fill_null(0) +
     pl.col("mtarjeta_master_consumo").fill_null(0)).alias("mtarjeta_total_consumo"),
    
    (pl.col("minversion1_pesos").fill_null(0) +
     pl.col("minversion1_dolares").fill_null(0)).alias("minversion1"),
    
    (pl.col("mpayroll").fill_null(0) +
     pl.col("mpayroll2").fill_null(0)).alias("mpayroll_todos"),
    
    (pl.col("ctarjeta_visa_debitos_automaticos").fill_null(0) +
     pl.col("ctarjeta_master_debitos_automaticos").fill_null(0)).alias("ctarjeta_todas_debitos_automaticos"),
    
    (pl.col("mttarjeta_visa_debitos_automaticos").fill_null(0) +
     pl.col("mttarjeta_master_debitos_automaticos").fill_null(0)).alias("mtarjeta_todas_debitos_automaticos"),
    
    (pl.col("ctarjeta_visa_descuentos").fill_null(0) +
     pl.col("ctarjeta_master_descuentos").fill_null(0)).alias("ctarjeta_todas_descuentos"),
    
    (pl.col("mtarjeta_visa_descuentos").fill_null(0) +
     pl.col("mtarjeta_master_descuentos").fill_null(0)).alias("mtarjeta_todas_descuentos"),
    
    (pl.col("ccomisiones_mantenimiento").fill_null(0) +
     pl.col("ccomisiones_otras").fill_null(0)).alias("comisiones"),
    
    (pl.col("mcomisiones_mantenimiento").fill_null(0) +
     pl.col("mcomisiones_otras").fill_null(0)).alias("mcomisiones_suma"),
    
    (pl.col("ctransferencias_recibidas").fill_null(0) +
     pl.col("ctransferencias_emitidas").fill_null(0)).alias("ctransferencias"),
    
    (pl.col("mtransferencias_recibidas").fill_null(0) +
     pl.col("mtransferencias_emitidas").fill_null(0)).alias("mtransferencias"),
    
    (pl.col("Master_msaldopesos").fill_null(0) +
     pl.col("Visa_msaldopesos").fill_null(0)).alias("tarjetas_msaldopesos"),
    
    (pl.col("Master_msaldodolares").fill_null(0) +
     pl.col("Visa_msaldodolares").fill_null(0)).alias("tarjetas_msaldodolares"),
    
    (pl.col("Master_mconsumospesos").fill_null(0) +
     pl.col("Visa_mconsumospesos").fill_null(0)).alias("tarjetas_mconsumospesos"),
    
    (pl.col("Master_mconsumosdolares").fill_null(0) +
     pl.col("Visa_mconsumosdolares").fill_null(0)).alias("tarjetas_mconsumosdolares"),
    
    (pl.col("Master_mlimitecompra").fill_null(0) +
     pl.col("Visa_mlimitecompra").fill_null(0)).alias("tarjetas_mlimitecompra"),
    
    (pl.col("Master_madelantopesos").fill_null(0) +
     pl.col("Visa_madelantopesos").fill_null(0)).alias("tarjetas_madelantopesos"),
    
    (pl.col("Master_madelantodolares").fill_null(0) +
     pl.col("Visa_madelantodolares").fill_null(0)).alias("tarjetas_madelantodolares"),
    
    (pl.col("Master_mpagado").fill_null(0) +
     pl.col("Visa_mpagado").fill_null(0)).alias("tarjetas_mpagado"),
    
    (pl.col("Master_mpagospesos").fill_null(0) +
     pl.col("Visa_mpagospesos").fill_null(0)).alias("tarjetas_mpagospesos"),
    
    (pl.col("Master_mpagosdolares").fill_null(0) +
     pl.col("Visa_mpagosdolares").fill_null(0)).alias("tarjetas_mpagosdolares"),
    
    (pl.col("Master_mconsumototal").fill_null(0) +
     pl.col("Visa_mconsumototal").fill_null(0)).alias("tarjetas_mconsumototal"),
    
    (pl.col("Master_cconsumos").fill_null(0) +
     pl.col("Visa_cconsumos").fill_null(0)).alias("tarjetas_cconsumos"),
    
    (pl.col("Master_cadelantosefectivo").fill_null(0) +
     pl.col("Visa_cadelantosefectivo").fill_null(0)).alias("tarjetas_cadelantosefectivo"),
    
    (pl.col("Master_mpagominimo").fill_null(0) +
     pl.col("Visa_mpagominimo").fill_null(0)).alias("tarjetas_mpagominimo"),
    
    (pl.col("Master_mfinanciacion_limite").fill_null(0) +
     pl.col("Visa_mfinanciacion_limite").fill_null(0)).alias("tarjetas_mfinanciacion_limite")
])

df = df.with_columns([
    (pl.col("mcuenta_debitos_automaticos").fill_null(0) +
     pl.col("mtarjeta_total_consumo").fill_null(0)).alias("consumos_mas_DA")
])


In [23]:
# columnas creadas en el paso anterior
creadas = [
    "suma_alt","cseguro_todos","matm_total","tarjeta_msaldototal","map_margen",
    "cc_ca","ctarjeta_total","ctarjeta_total_transacciones","mtarjeta_total_consumo","minversion1",
    "mpayroll_todos","consumos_mas_DA","ctarjeta_todas_debitos_automaticos","mtarjeta_todas_debitos_automaticos","ctarjeta_todas_descuentos",
    "mtarjeta_todas_descuentos","comisiones","mcomisiones_suma","ctransferencias","mtransferencias",
    "tarjetas_msaldopesos","tarjetas_msaldodolares","tarjetas_mconsumospesos","tarjetas_mconsumosdolares","tarjetas_mlimitecompra",
    "tarjetas_madelantopesos","tarjetas_madelantodolares","tarjetas_mpagado","tarjetas_mpagospesos",
    "tarjetas_mpagosdolares","tarjetas_mconsumototal","tarjetas_cconsumos","tarjetas_cadelantosefectivo",
    "tarjetas_mpagominimo","tarjetas_mfinanciacion_limite"
]

todas = originales + creadas

In [24]:
# lag 1
df = df.with_columns([
    pl.col(col)
    .sort_by(pl.col("foto_mes")) 
    .shift(1)
    .over("numero_de_cliente") 
    .alias(f"{col}_lag1")
    for col in todas
])

# lag 2
df = df.with_columns([
    pl.col(col)
    .sort_by(pl.col("foto_mes")) 
    .shift(2)
    .over("numero_de_cliente") 
    .alias(f"{col}_lag2")
    for col in todas
])

In [25]:
# delta 1
df = df.with_columns([
    (pl.col(col) - pl.col(f"{col}_lag1")).alias(f"{col}_delta1")
    for col in todas
])

# delta 2
df = df.with_columns([
    (pl.col(col) - pl.col(f"{col}_lag2")).alias(f"{col}_delta2")
    for col in todas
])

In [26]:
# moving average de lag-1  lag-2, para tener valor si alguno de los 2 es null
df = df.with_columns([
    pl.col(col)
    .sort_by(pl.col("foto_mes")) 
    .shift(1)  # para excluir el mes corriente
    .rolling_mean(window_size=2, min_periods=1)
    .over("numero_de_cliente") 
    .alias(f"{col}_ma2_lag")
    for col in todas
])

# delta
df = df.with_columns([
    (pl.col(col) - pl.col(f"{col}_ma2_lag")).alias(f"{col}_delta_ma2")
    for col in todas
])

In [27]:
dataset_path

'/home/eanegrin/datasets/'

In [28]:
df.write_parquet(dataset_path + 'competencia_03_fe_v05.parquet')

In [29]:
!gsutil cp /home/eanegrin/datasets/competencia_03_fe_v05.parquet /home/eanegrin/buckets/b1/datasets/

Copying file:///home/eanegrin/datasets/competencia_03_fe_v05.parquet...
- [1 files][  9.5 GiB/  9.5 GiB]                                                
Operation completed over 1 objects/9.5 GiB.                                      


In [ ]:
# Dataset con solo periodo de test y de prediccion para que nos de la RAM para levantarlo localmente
df = df.filter(pl.col("foto_mes").is_in([202107,202109]))

df.write_parquet(dataset_path + 'competencia_03_fe_v05_predict.parquet')

In [ ]:
!gsutil cp /home/eanegrin/datasets/competencia_03_fe_v05_predict.parquet /home/eanegrin/buckets/b1/datasets/